In [1]:
from sklearn import tree
from pychord_tools.models import load_model
import json
import glob, os,sys

sys.path.append('../')

In [2]:
songs = ['Lily','Mountain','Century','Where','Hole','Train']

anno = ['../test_data/Lily Was Here/Lily Was Here.json',
        '../test_data/Mountain At My Gates/Mountain At My Gates.json',
        '../test_data/20th Century Boy/20th Century Boy.json',
        '../test_data/Where Did You Sleep Last Night/Where Did You Sleep Last Night.json',
        '../test_data/Hole In My Shoe/Hole In My Shoe.json', 
        '../test_data/Runaway Train/Runaway Train.json']  

ex_2_id={0:'../test_data/Lily Was Here/Lily Was Here.json',
         1:'../test_data/Mountain At My Gates/Mountain At My Gates.json',
         2:'../test_data/20th Century Boy/20th Century Boy.json',
         3:'../test_data/Where Did You Sleep Last Night/Where Did You Sleep Last Night.json',
         4:'../test_data/Hole In My Shoe/Hole In My Shoe.json',
         5:'../test_data/Runaway Train/Runaway Train.json'}

## Move files (Temporary)

In [5]:
filter_in = ['Small_04','Small_05','Small_tiled_room','h008','Living_Room','Medium_02','Medium_05',
            'h027','Large_05','Large_tiled_room','Large_wooden_room','h094']

In [7]:
import shutil

path_data = '/media/eduard/Maxtor/TFG_DATABASE_AUGMENTATION/MIT_Reverbs/'
store_path = '../Databases/Augmented_database/MIT_Reverbs/'

files = glob.glob(path_data + '*.wav')
filtered_files = []
# mask = ['Laptop_Speaker','Mobile_Phone_01','Mobile_Phone_03','Radio_01','Radio_02','Voice_Recorder'] #MeldaBox
# mask = ['AKG_C3000_25cm','Sennheiser_e609_Direct','Shure_Beta52_Direct','TakStar_Direct_A']
for f in files:
    if any(ext in f for ext in filter_in):
#     if any(s in f for s in mask):
#         shutil.copy(f ,store_path )
        shutil.copy(f ,store_path)

# Change file names for interpretibility
    

In [8]:
names_map = {'Laptop_Speaker':'SimComp',
            'Mobile_Phone_01' : 'SimMobile1',
            'Mobile_Phone_03' : 'SimMobile2',
            'Radio_01':'SimRadio1',
            'Radio_02' : 'SimRadio2',
            'Voice_Recorder_01':'SimVoiceRecorder',
            'AKG_C3000_25cm':'AKG',
            'Sennheiser_e609_Direct':'Sennheiser',
            'Shure_Beta52_Direct':'Shure',
            'TakStar_Direct_A' : 'TakStar',
             
            'Small_04':'Small1',
            'Small_05': 'Small2',
            'Small_tiled_room':'Small3',
            'h008':'Small4',
             
            'Living_Room':'Medium1',
            'Medium_02':'Medium2',
            'Medium_05':'Medium3',
            'h027':'Medium4',
             
             'Large_05':'Large1',
             'Large_tiled_room':'Large2',
             'Large_wooden_room':'Large3',
             'h094':'Large4'}

In [9]:
import json

paths = ['../Databases/Augmented_database/MIT_Reverbs/','../Databases/Augmented_database/MeldaProduction_Rooms/']

for p in paths:
    files = glob.glob(p + '*.wav')
    for f in files:
        f_name = f.split('/')[-1].replace('.wav','').split('_')
        ir =  '_'.join(f_name[4:])
        f_head = '_'.join(f_name[:3])

        if ir in names_map.keys():
        
            out_name = p + f_head+ '_' + names_map[ir]+'.wav'
        else:
            out_name = p + f_head+ '_' + ir+'.wav'
        print(out_name)
        os.rename(f,out_name)

../Databases/Augmented_database/MIT_Reverbs/Where_Eastman_100_Medium4.wav
../Databases/Augmented_database/MIT_Reverbs/Century_Ibanez_130_Medium4.wav
../Databases/Augmented_database/MIT_Reverbs/Century_Larrivee_130_Large4.wav
../Databases/Augmented_database/MIT_Reverbs/Mountain_Larrivee_104_Large4.wav
../Databases/Augmented_database/MIT_Reverbs/Hole_Larrivee_102_Large4.wav
../Databases/Augmented_database/MIT_Reverbs/Where_Larrivee_100_Large4.wav
../Databases/Augmented_database/MIT_Reverbs/Century_Eastman_130_Large4.wav
../Databases/Augmented_database/MIT_Reverbs/Century_Eastman_130_Small4.wav
../Databases/Augmented_database/MIT_Reverbs/Hole_Telecaster_102_Large4.wav
../Databases/Augmented_database/MIT_Reverbs/Train_Eastman_112_Large4.wav
../Databases/Augmented_database/MIT_Reverbs/Lily_Eastman_108_Large4.wav
../Databases/Augmented_database/MIT_Reverbs/Hole_Epiphone_102_Small4.wav
../Databases/Augmented_database/MIT_Reverbs/Train_Telecaster_112_Medium4.wav
../Databases/Augmented_database

### Elemets from different databases

In [22]:
directories_filter = ['Raw_database/']
file_name = 'Raw_database_with_third_intervals'

path_data = '../Databases/'
loader_path = '../Databases/Loaders/'

In [23]:
def filter_out( files, filter_out = None):
    """
    Filter out files which do not want to load
    """
    filtered_files = []
    if filter_out:
        for f in files:
            if not any(ext in f for ext in filter_out):              
                filtered_files.append(f )
        return filtered_files
    else:
        return files

In [24]:
#Load files from databases
files = []
for d in directories_filter:
    files.append(glob.glob(path_data + d +'*.wav'))
        
files = [item for sublist in files for item in sublist]

files = filter_out(files, filter_out = ['Lily','Century','Where','Hole','Train'])

#Build json file with paths to the files for later laoding them and calculate chromas
count = 0
file_list = []

for f in files:
    info = {'id':0,
       'path':'',
       'exercise_id':'',
       'latency' :0.0,
        'name' : ''}
    
    name = f.replace(path_data,'')
    if 'Augmented_database' in name:
        name = name.split('/')[2]
    else:
        name = name.split('/')[1]
    info['id'] = count
    info['path'] = f.replace('../Databases/','')
    info['name'] = name
    song = name.split('_')[0]

    info['exercise_id'] = [i for i,f in enumerate(songs) if f == song][0]

    file_list.append(info)
    count += 1
    
#Store json file in Databases/Loaders/ folder
jsonString = json.dumps(file_list)
jsonFile = open(loader_path + file_name + ".json", "w")
jsonFile.write(jsonString)
jsonFile.close()